# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import time
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD

import pickle

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\osama\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\osama\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\osama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///CleanedDisasterResponse.db')
df = pd.read_sql_table("disaster_messages", con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df.message
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [4]:
X.to_csv('messgaes.csv')
# After examining the messages, I have noticed that some of them have URL in them, which means it should be replaced

In [5]:
def tokenize(text):
    # Replace URL with urlplaceholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # Normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [token for token in tokens if token not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultiOutputClassifier(tree.DecisionTreeClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
start = time.time()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)
print("Finished Training in: %s minutes " % ((time.time() - start)/60))

Finished Training in: 4.591029949982961 minutes 


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# predict
def class_report(model, X_test, Y_test):
    Y_pred = model.predict(X_test)
    for i, col in enumerate(Y_test):
        print(col)
        print(classification_report(Y_test[col], Y_pred[:, i]))

In [9]:
start = time.time()
class_report(pipeline, X_test, Y_test)
Y_pred = pipeline.predict(X_test)
print("Finished Testing in: %s minutes " % ((time.time() - start)/60))

related
              precision    recall  f1-score   support

           0       0.53      0.51      0.52      1528
           1       0.85      0.85      0.85      4981
           2       0.13      0.38      0.20        45

    accuracy                           0.77      6554
   macro avg       0.51      0.58      0.52      6554
weighted avg       0.78      0.77      0.77      6554

request
              precision    recall  f1-score   support

           0       0.91      0.92      0.91      5428
           1       0.58      0.55      0.56      1126

    accuracy                           0.85      6554
   macro avg       0.74      0.73      0.74      6554
weighted avg       0.85      0.85      0.85      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

### 6. Improve your model
Use grid search to find better parameters. 

pipeline.get_params()

In [10]:
parameters =  {'clf__estimator__splitter': ['best', 'random'], 
              'clf__estimator__min_samples_split': [2, 4],
              } 

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
start = time.time()
cv.fit(X_train, Y_train)
print("Finished Training in: %s minutes " % ((time.time() - start)/60))
start = time.time()
class_report(cv, X_test, Y_test)
print("Finished Testing in: %s minutes " % ((time.time() - start)/60))

Finished Training in: 64.12116165161133 minutes 
related
              precision    recall  f1-score   support

           0       0.54      0.50      0.52      1528
           1       0.85      0.86      0.86      4981
           2       0.13      0.36      0.19        45

    accuracy                           0.77      6554
   macro avg       0.51      0.57      0.52      6554
weighted avg       0.78      0.77      0.77      6554

request
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      5428
           1       0.60      0.53      0.56      1126

    accuracy                           0.86      6554
   macro avg       0.75      0.73      0.74      6554
weighted avg       0.85      0.86      0.85      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           0.99      6554
   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
# Using SVC instead of DecisionTreeClassifier  
pipeline2 = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultiOutputClassifier(SVC()))
])
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000023944DFA950>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                       class_weight=None, coef0=0.0,
                                       decision_function_shape='ovr', degree=3,
                                       gamma='scale', kernel='rbf', max_iter=-1,
                                       probability=False

In [13]:
parameters2 = {
    'clf__estimator__C': [1, 10],
    'clf__estimator__kernel': ['linear', 'rbf']

}
cv2 = GridSearchCV(pipeline2, param_grid=parameters2)

start = time.time()
cv2.fit(X_train, Y_train)
print("Finished Training in: %s minutes " % ((time.time() - start)/60))

start = time.time()
class_report(cv2, X_test, Y_test)
print("Finished Testing in: %s minutes " % ((time.time() - start)/60))

C:\Users\osama\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: The number of classes has to be greater than one; got 1 class

  FitFailedWarning)
C:\Users\osama\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: The number of classes has to be greater than one; got 1 class

  FitFailedWarning)
C:\Users\osama\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: The number of classes has to be greater than one; got 1 class

  FitFailedWarning)
C:\Users\osama\Anaconda3\lib\site-packages\sklearn\model_selecti

ValueError: The number of classes has to be greater than one; got 1 class

### 9. Export your model as a pickle file

In [ ]:
# Saving the model to pickle file
joblib.dump(pipeline, 'SVC.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.